In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time

chrome_path = r"C:\chromedriver.exe"


options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(chrome_path), options=options)


url = "https://www.cricbuzz.com/cricket-scorecard-archives"
driver.get(url)
time.sleep(3)

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

series_data = []

years = soup.select(".cb-schdl-hdr, .cb-arcv-yr")
series_blocks = soup.select(".cb-srs-cat, .cb-srs-lst-itm")

current_year = None
current_type = None

for block in series_blocks:
    if "cb-arcv-yr" in block.get("class", []):
        current_year = block.text.strip()
    elif "cb-srs-cat" in block.get("class", []):
        current_type = block.text.strip()
    elif "cb-srs-lst-itm" in block.get("class", []):
        anchor = block.find("a")
        if anchor:
            series_name = anchor.text.strip()
            series_href = "https://www.cricbuzz.com" + anchor.get("href", "")
            series_data.append({
                "year": current_year,
                "series_type": current_type,
                "series_name": series_name,
                "series_href": series_href
            })

df = pd.DataFrame(series_data)
print(df.head())
df.to_csv("cricbuzz_series_data.csv", index=False)

   year    series_type                                   series_name  \
0  None  International       Malaysia Quadrangular T20I Series, 2025   
1  None  International             Zimbabwe tour of Bangladesh, 2025   
2  None  International              The North American T20 Cup, 2025   
3  None  International  Central American Cricket Championships, 2025   
4  None  International      Hong Kong T20I Quadrangular Series, 2025   

                                         series_href  
0  https://www.cricbuzz.com/cricket-series/9710/m...  
1  https://www.cricbuzz.com/cricket-series/9514/z...  
2  https://www.cricbuzz.com/cricket-series/9677/t...  
3  https://www.cricbuzz.com/cricket-series/9692/c...  
4  https://www.cricbuzz.com/cricket-series/9647/h...  


In [11]:
df = pd.read_csv('cricbuzz_series_data.csv')
df.columns



Index(['year', 'series_type', 'series_name', 'series_href'], dtype='object')

In [12]:

df['series_name'] = df['series_name'].astype(str)
df['year'] = df['series_name'].str.extract(r',\s*(\d{4})')
df['series_name'] = df['series_name'].str.replace(r',\s*\d{4}', '', regex=True)

df.to_csv('cricbuzz_series_data.csv', index=False)
